## Código para identificar tiendas Cornershop por código postal

Con base en los códigos postales de SEPOMEX, se identifican las tiendas Cornershop que se encuentran en cada uno de los códigos postales.

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import time
import requests
import random

In [4]:
#Directorio de trabajo
#os.chdir('C:/Users/claudio.pacheco/Documents/GitHub/pacic/')
os.chdir("D:/")

#### Descarga de datos de Sepomex

In [5]:
#Descarga de códigos postales de Ciudad de México
url = 'https://www.correosdemexico.gob.mx/SSLServicios/ConsultaCP/CodigoPostal_Exportar.aspx'
driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)
combo = driver.find_element(By.CLASS_NAME, "cbo")
select = Select(combo)
select.select_by_visible_text("Ciudad de México")
#Descarga en carpeta de trabajo
driver.find_element(By.ID, "btnDescarga").click()
time.sleep(5)
driver.close()


C:\Users\claud\AppData\Local\Temp\ipykernel_17192\1557182785.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/chromedriver.exe")


In [6]:

#Mover a carpeta de trabajo el archivo descargado

#os.rename('C:/Users/claudio.pacheco/Downloads/Ciudad de México.xls', 'C:/Users/claudio.pacheco/Documents/GitHub/pacic/cp/Ciudad de México.xls')

In [7]:
#Leer archivo descargado
cp = pd.read_excel('cp/Ciudad de México.xls',sheet_name="Distrito_Federal",usecols=["d_codigo", "d_asenta", 
                                                                                    "d_tipo_asenta", "D_mnpio", 
                                                                                    "c_mnpio","d_zona"],dtype={"d_codigo":str})

In [8]:
cp

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,c_mnpio,d_zona
0,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
1,01010,Los Alpes,Colonia,Álvaro Obregón,10,Urbano
2,01020,Guadalupe Inn,Colonia,Álvaro Obregón,10,Urbano
3,01030,Axotla,Pueblo,Álvaro Obregón,10,Urbano
4,01030,Florida,Colonia,Álvaro Obregón,10,Urbano
...,...,...,...,...,...,...
1517,16840,Santa Cruz Chavarrieta,Colonia,Xochimilco,13,Urbano
1518,16850,Chapultepec,Barrio,Xochimilco,13,Urbano
1519,16860,Santa Cruz de Guadalupe,Colonia,Xochimilco,13,Urbano
1520,16880,Santa Cecilia Tepetlapa,Pueblo,Xochimilco,13,Urbano


In [9]:
#Eliminar duplicados
cp = cp.drop_duplicates(subset=['d_codigo'], keep='first')
cp

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,c_mnpio,d_zona
0,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
1,01010,Los Alpes,Colonia,Álvaro Obregón,10,Urbano
2,01020,Guadalupe Inn,Colonia,Álvaro Obregón,10,Urbano
3,01030,Axotla,Pueblo,Álvaro Obregón,10,Urbano
5,01040,Campestre,Colonia,Álvaro Obregón,10,Urbano
...,...,...,...,...,...,...
1517,16840,Santa Cruz Chavarrieta,Colonia,Xochimilco,13,Urbano
1518,16850,Chapultepec,Barrio,Xochimilco,13,Urbano
1519,16860,Santa Cruz de Guadalupe,Colonia,Xochimilco,13,Urbano
1520,16880,Santa Cecilia Tepetlapa,Pueblo,Xochimilco,13,Urbano


### Búsqueda de tiendas Cornershop por código postal

In [10]:
#Función para obtener tiendas por código postal
def zip_code(zip_code:str):
    url="https://cornershopapp.com/api/v1/collections/13?locality="+zip_code+"&country=MX"
    r = requests.get(url)
    #Si no se encuentra, lanza una df en blanco
    if r.status_code == 404:
        df=pd.DataFrame({"id_tienda":[],"tienda":[],"codigo_postal":[]})
        return df
    #Caso contrario, lanza un df con las tiendas
    else:
        data_zip = r.json()
        tiendas=[]
        ids=[]
        codigo=zip_code
        for stores in data_zip["branches"]:
            tiendas.append(stores["name"])
            ids.append(stores["id"])
    #Llevarlo a un dataframe
    df=pd.DataFrame({"id_tienda":ids,"tienda":tiendas,"codigo_postal":codigo})
    return df

In [11]:
tiendascorner=pd.DataFrame()
for codigos in range(len(cp["d_codigo"])): 
    time.sleep(random.randint(1,2))
    codigo = cp["d_codigo"].get(codigos)
    if codigo is not None:
        tiendascorner = pd.concat([tiendascorner, zip_code(codigo)], axis=0)


In [12]:
tiendascorner

,id_tienda,tienda,codigo_postal
0,1445,Chedraui,01000
1,49,City Market,01000
2,92,Costco,01000
3,4414,La Comer,01000
4,10002,Soriana,01000
...,...,...,...
2,92,Costco,11950
3,4412,La Comer,11950
4,10362,Soriana,11950
5,4432,Fresko,11950


In [13]:
#pegar con datos de cp
tiendascorner = pd.merge(tiendascorner, cp, left_on='codigo_postal', right_on='d_codigo', how='left')

In [14]:
tiendascorner

,id_tienda,tienda,codigo_postal,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,c_mnpio,d_zona
0,1445,Chedraui,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
1,49,City Market,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
2,92,Costco,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
3,4414,La Comer,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
4,10002,Soriana,01000,01000,San Ángel,Colonia,Álvaro Obregón,10,Urbano
...,...,...,...,...,...,...,...,...,...
5474,92,Costco,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano
5475,4412,La Comer,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano
5476,10362,Soriana,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano
5477,4432,Fresko,11950,11950,Lomas Altas,Colonia,Miguel Hidalgo,16,Urbano


In [15]:
#Completar registros con información de cornershop
url_basica="https://cornershopapp.com/api/v3/branches/"
url_fin="?with_suspended_slots&app_capabilities=DATA_REQUEST_BASIC_FORM,CUSTOMER_WITH_SENDBIRD_CHAT,DATA_REQUEST_PRESCRIPTION,LOYALTY_PROVIDERS_WITH_2FA,IMAGE_SUPPORT_CHAT,TC_BLOCKER,CUSTOMER_SMART_SERVICE_FEE,STORE_WITHOUT_PRICING_NOTES"

In [ ]:
https://cornershopapp.com/api/v3/branches/46322/?with_suspended_slots&app_capabilities=DATA_REQUEST_BASIC_FORM,CUSTOMER_WITH_SENDBIRD_CHAT,DATA_REQUEST_PRESCRIPTION,LOYALTY_PROVIDERS_WITH_2FA,IMAGE_SUPPORT_CHAT,TC_BLOCKER,CUSTOMER_SMART_SERVICE_FEE,STORE_WITHOUT_PRICING_NOTES

In [22]:
id_tienda

[1445,
 49,
 92,
 4414,
 10002,
 4434,
 4423,
 1445,
 49,
 92,
 4417,
 10353,
 4434,
 4423,
 1445,
 49,
 92,
 4417,
 10353,
 4434,
 4423,
 138,
 58,
 92,
 4417,
 10353,
 4434,
 4423,
 1445,
 49,
 92,
 4417,
 10353,
 4434,
 4423,
 1445,
 49,
 92,
 4417,
 10002,
 4434,
 4423,
 1445,
 49,
 92,
 4414,
 10353,
 4434,
 4423,
 1445,
 49,
 92,
 4417,
 10002,
 4434,
 4423,
 1445,
 49,
 92,
 4414,
 10002,
 4434,
 4423,
 1445,
 49,
 92,
 4413,
 10002,
 4434,
 4423,
 1445,
 49,
 92,
 4413,
 10002,
 4434,
 4423,
 1445,
 49,
 92,
 4414,
 10002,
 4434,
 4423,
 46322,
 54,
 92,
 4417,
 10353,
 4432,
 4418,
 46322,
 54,
 92,
 4417,
 10353,
 4432,
 4418,
 46322,
 54,
 92,
 4417,
 10353,
 4432,
 4419,
 46322,
 54,
 92,
 4417,
 10362,
 4432,
 4419,
 46322,
 54,
 92,
 4417,
 10362,
 4432,
 4419,
 46322,
 54,
 92,
 4417,
 10353,
 4432,
 4419,
 46322,
 54,
 92,
 4417,
 10362,
 4432,
 4419,
 138,
 54,
 92,
 4417,
 10362,
 4432,
 4418,
 46322,
 54,
 92,
 4417,
 10353,
 4432,
 4418]

In [30]:
id_tienda=[]
sucursales=[]
latitud=[]
longitud=[]
store_name=[]
store_id=[]
address=[]


#buscar los id_tiendas de tiendascorner en la api de cornershop
for tiendas in range(len(tiendascorner["id_tienda"])):
    time.sleep(random.randint(1,2))
    url = url_basica+str(tiendas)+url_fin
    r = requests.get(url)
    if r.status_code == 404:
        id_tienda.append("NA")
        sucursales.append("NA")
        latitud.append("NA")
        longitud.append("NA")
        store_name.append("NA")
        store_id.append("NA")
        address.append("NA")
    else:
        data = r.json()
        id_tienda.append(data["branch"]["id"])
        sucursales.append(data["branch"]["name"])
        latitud.append(data["branch"]["lat"])
        longitud.append(data["branch"]["lng"])
        store_name.append(data["branch"]["store_name"])
        store_id.append(data["branch"]["store_id"])
        address.append(data["branch"]["address"])


In [ ]:
#pasa los datos a un dataframe
df=pd.DataFrame({"id_sucursal":id_tienda,"sucursal":sucursales,"latitud":latitud,"longitud":longitud,"store_name":store_name,"store_id":store_id,"address":address})

In [ ]:
#Pegar con tiendascorner
tiendascorner_final = pd.merge(tiendascorner, df, left_on='id_tienda', right_on='id_sucursal', how='left')

In [ ]:
#Guardar en excel
tiendascorner_final.to_excel("tiendas_corner.xlsx",index=False)